In [1]:
%pip install pandas
%pip install scikit-learn
%pip install request
%pip install matplotlib
%pip install numpy
%pip install joblib

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement request (from versions: none)
ERROR: No matching distribution found for request


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import mlflow
from mlflow.models.signature import infer_signature
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


# create a dataframe with data celsius and fahrenheit
df = pd.DataFrame({'celsius': [-40, -10, 0, 8, 15, 22, 38],
                     'fahrenheit': [-40, 14, 32, 46, 59, 72, 100]})

# show info
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   celsius     7 non-null      int64
 1   fahrenheit  7 non-null      int64
dtypes: int64(2)
memory usage: 244.0 bytes


In [8]:
# params for linear regression
params = {
    'fit_intercept': True,
    'positive': True,
    'copy_X': True,
    'n_jobs': 10
}

# create a linear regression model
lr = LinearRegression(**params)

# prepare data
X = df[['celsius']].values.reshape(-1, 1)
Y = df[['fahrenheit']].values

# Split data
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=50)

# Train model
lr.fit(x_train, y_train)

# Predict
y_pred = lr.predict([[89]])

print(y_pred)


[[191.61628726]]


In [9]:
# export model

import joblib

joblib.dump(lr, '../streamlit/model/temp.pkl')

['../streamlit/model/temp.pkl']

In [11]:

mlflow.set_tracking_uri('http://localhost:8080')
mlflow.set_experiment('Temperature')

with mlflow.start_run():

    mlflow.log_input(mlflow.data.from_pandas(df))
    mlflow.log_param('fit_intercept', params['fit_intercept'])
    mlflow.log_param('positive', params['positive'])
    mlflow.log_param('copy_X', params['copy_X'])
    mlflow.log_param('n_jobs', params['n_jobs'])

    mlflow.log_metric('mean_squared_error', mean_squared_error(y_test, lr.predict(x_test)))
    mlflow.log_metric('mean_absolute_error', mean_absolute_error(y_test, lr.predict(x_test)))
    mlflow.log_metric('r2_score', r2_score(y_test, lr.predict(x_test)))

    signature = infer_signature(x_train, lr.predict(x_train))
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="temperature_model",
        signature=signature,
        input_example=x_train,
        registered_model_name="tracking-temp-model",
    )
    mlflow.log_artifact('../streamlit/model/temp.pkl')
    

c:\Users\diogenes monegro\AppData\Local\Programs\Python\Python311\Lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
Successfully registered model 'tracking-temp-model'.
2024/09/03 09:56:58 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracki

2024/09/03 09:56:59 INFO mlflow.tracking._tracking_service.client: 🏃 View run dashing-swan-440 at: http://localhost:8080/#/experiments/547292408523564591/runs/b204c0e9212c4f7ca6346ca36ac1a713.
2024/09/03 09:56:59 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:8080/#/experiments/547292408523564591.
